In [1]:
import tensorflow as tf
import core.utils as utils
from tensorflow.python.saved_model import tag_constants
import cv2
import numpy as np
import os
import pathlib

import PIL
from PIL import Image, ImageDraw, ImageFont

import matplotlib
import matplotlib.pyplot as plt
import io
import scipy.misc

ImportError: DLL load failed: 지정된 프로시저를 찾을 수 없습니다.

In [ ]:
MODEL_PATH='./checkpoints/yolov4-416'
IOU_THRESHOLD = 0.45 #IOU가능 수치
SCORE_THRESHOLD = 0.25 #버릴수치
INPUT_SIZE = 416
FOLDER_PATH="/home/lab435/Desktop/sw/PythonStudy/tensorflow-yolov4-tflite/data/GO01_Rename" #사진폴더 경로

In [ ]:
#LOAD MODEL
saved_model_loaded = tf.saved_model.load(MODEL_PATH, tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']

In [ ]:
def ref_clsname(cls_array,scr_array,cls_cnt):
    print("Object - ",cls_cnt[0])
    inani=0
    for idx_num in range(0,cls_cnt[0]):
        if(cls_array[0][idx_num] != 0):

            class_txt = open("./data/classes/coco.names","r")
            data = class_txt.read().split('\n')[int(cls_array[0][idx_num])]
            predic_p=scr_array[0][idx_num]*100
            print("- %d%%"%predic_p,data)
            
            class_txt.close()
            
            if(cls_array[0][idx_num]<14 or cls_array[0][idx_num] >24):inani+=1 #동물범위조정
            
    if(cls_cnt[0]==0 or cls_cnt==inani): #inanimate count == object count and non object
        print("NotFound")
        return "/NotFound/"
    else:
        print("Found")
        return "/Found/"

In [ ]:
def main(img_path,img_name):
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #OpenCV는 RGB가 아닌 BFR형식

    img_input = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
    img_input = img_input / 255.
    img_input = img_input[np.newaxis, ...].astype(np.float32)
    img_input = tf.constant(img_input)

    pred_bbox = infer(img_input)

    for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]

    boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
        boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape(
            pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
        max_output_size_per_class=50,
        max_total_size=50,
        iou_threshold=IOU_THRESHOLD,
        score_threshold=SCORE_THRESHOLD
    )

    pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]
    result = utils.draw_bbox(img, pred_bbox)
    
    result = cv2.cvtColor(np.array(result), cv2.COLOR_RGB2BGR)
    OUTPUT_PATH = FOLDER_PATH+ref_clsname(classes.numpy(), scores.numpy(), valid_detections.numpy())+img_name
    cv2.imwrite(OUTPUT_PATH, result)

In [ ]:
if __name__ == '__main__':
    INPUT_PATH = pathlib.Path(FOLDER_PATH)
    IMAGE_PATH_LIST = list(INPUT_PATH.glob('*.JPG')) #경로의 폴더 하위 .jpg 파일 모두 가져와서 리스트로 만들기
    path_list_count = 1
    for IMAGE_PATH in IMAGE_PATH_LIST: #만든 리스트의 .jpg파일을 각각 분류
        print("------------------\n",IMAGE_PATH.parts[-1],"(",path_list_count,"/",len(IMAGE_PATH_LIST),")")
        main(IMAGE_PATH.absolute().as_posix(),IMAGE_PATH.parts[-1]) #분류시작
        path_list_count += 1